In [2]:
from openpyxl import load_workbook

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
from Concrete.Import.ExcelData import ExcelData
from Concrete.PathCalculator import PathCalculator
from Concrete.Routing.Step import Step

In [3]:
def print_hi(name):
    # Use a breakpoint in the code line below to debug your script.
    print(f'Hi, {name}')  # Press Ctrl+F8 to toggle the breakpoint.

def ReadExcelSheet(filePath, sheetIndex):
    workbook = load_workbook(filename=filePath)
    sheet = workbook[workbook.sheetnames[sheetIndex]]
    return sheet


In [4]:
metersSheet = ReadExcelSheet("Files/Products.xlsx", 1)
physicalDistanceBetweenLinesInMeter = int(metersSheet["B1"].value)
physicalDistanceBetweenStepsInMeter = int(metersSheet["B2"].value)
lineChargeAddressHeight = int(metersSheet["B3"].value) / 4

physicalDistanceBetweenLinesInMeter, physicalDistanceBetweenStepsInMeter, lineChargeAddressHeight

(5, 1, 15.0)

In [5]:
## get excelDatas
productsSheet = ReadExcelSheet("Files/Products.xlsx", 0)
excelDatas = []
for i in range(productsSheet.max_row):
    if i == 0:
        continue

    row_address = productsSheet[("A" + str(i + 1))].value
    if row_address == None or row_address == "":
        continue
    row_product_no = productsSheet[("C" + str(i + 1))].value
    row_category = productsSheet[("E" + str(i + 1))].value

    excelData = ExcelData.GetWithAddress(row_address, row_product_no, row_category)
    excelDatas.append(excelData)

print(excelDatas[33])

ProductNo: 34, Line: 11, BlockY: A, ChargeAddress: 55, BlockX: B, Category: NF,


In [6]:
# get orderedProductNos
ordersSheet = ReadExcelSheet("Files/Orders.xlsx", 0)
orderedProductNos = []
for i in range(ordersSheet.max_row):
    if i == 0:
        continue

    productNo = ordersSheet[("J" + str(i + 1))].value
    orderedProductNos.append(productNo)
    
    
print(orderedProductNos), len(orderedProductNos)

[1, 2, 3, 6, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 34]


(None, 26)

In [7]:
## get orders

orders = []
for i in range(len(excelDatas)):
    excelData = excelDatas[i]

    found = False
    for j in range(len(orderedProductNos)):
        if orderedProductNos[j] == excelData.ProductNo:
            found = True
            break

    if found == True:
        orders.append(excelData)
        
print(orders[-1])

ProductNo: 34, Line: 11, BlockY: A, ChargeAddress: 55, BlockX: B, Category: NF,


In [8]:
# get pickers

pickersSheet = ReadExcelSheet("Files/Orders.xlsx", 1)
pickers = []
for i in range(pickersSheet.max_row):
    if i == 0:
        continue

    picker_name = pickersSheet[("A" + str(i + 1))].value
    categories = str(pickersSheet[("B" + str(i + 1))].value).split(",")
    targetLine = int(pickersSheet[("C" + str(i + 1))].value)

    pickers.append({
        "Name": picker_name,
        "Categories": categories,
        "TargetLine": targetLine,
        "Steps": {
            "SShape": [],
            "MidPoint": [],
            "LargestGap": []
        },
        "Distances": {
            "SShape": [],
            "MidPoint": [],
            "LargestGap": []
        }
    })

print(pickers)

[{'Name': 'Gündüz', 'Categories': ['B1', 'GD1'], 'TargetLine': 2, 'Steps': {'SShape': [], 'MidPoint': [], 'LargestGap': []}, 'Distances': {'SShape': [], 'MidPoint': [], 'LargestGap': []}}, {'Name': 'Kardelen', 'Categories': ['D1', 'D2'], 'TargetLine': 5, 'Steps': {'SShape': [], 'MidPoint': [], 'LargestGap': []}, 'Distances': {'SShape': [], 'MidPoint': [], 'LargestGap': []}}, {'Name': 'Mustafa', 'Categories': ['DG1', 'NF'], 'TargetLine': 8, 'Steps': {'SShape': [], 'MidPoint': [], 'LargestGap': []}, 'Distances': {'SShape': [], 'MidPoint': [], 'LargestGap': []}}, {'Name': 'Arda', 'Categories': ['GD2', 'KOZ'], 'TargetLine': 7, 'Steps': {'SShape': [], 'MidPoint': [], 'LargestGap': []}, 'Distances': {'SShape': [], 'MidPoint': [], 'LargestGap': []}}]


In [10]:

for i in range(len(pickers)):
    picker = pickers[i]

    pickerOrders = []

    for j in range(len(orders)):
        order = orders[j]

        found = False
        for k in range(len(picker["Categories"])):
            category = picker["Categories"][k]

            if order.Category == category:
                found = True
                break

        if found == True:
            pickerOrders.append(order)

    print(picker["Name"] + " " + str(len(pickerOrders)) + " adet ürün toplayacaktır.")
    print("")

    print("Ürünler:")
    for i in range(len(pickerOrders)):
        print("-> Ürün " + str(i + 1) + " = ", pickerOrders[i])
    picker["Steps"]["SShape"] = PathCalculator.GetDistance(pickerOrders, lineChargeAddressHeight, 1, 0, picker["TargetLine"], "SShape")["Steps"]
    picker["Steps"]["MidPoint"] = PathCalculator.GetDistance(pickerOrders, lineChargeAddressHeight, 1, 0, picker["TargetLine"], "MidPoint")["Steps"]
    picker["Steps"]["LargestGap"] = PathCalculator.GetDistance(pickerOrders, lineChargeAddressHeight, 1, 0, picker["TargetLine"], "LargestGap")["Steps"]

    print("")
    sshapeDistance = Step.GetTotalDistance(physicalDistanceBetweenStepsInMeter, physicalDistanceBetweenLinesInMeter, picker["Steps"]["SShape"])

    print("SShape algoritmasına göre gitmesi gereken rota:")
    for i in range(len(picker["Steps"]["SShape"])):
        print("-> Adım " + str(i + 1) + " = ", picker["Steps"]["SShape"][i])

    print("")
    midpointDistance = Step.GetTotalDistance(physicalDistanceBetweenStepsInMeter, physicalDistanceBetweenLinesInMeter, picker["Steps"]["MidPoint"])

    print("MidPoint algoritmasına göre gitmesi gereken rota:")
    for i in range(len(picker["Steps"]["MidPoint"])):
        print("-> Adım " + str(i + 1) + " = ", picker["Steps"]["MidPoint"][i])

    print("")
    largestgapDistance = Step.GetTotalDistance(physicalDistanceBetweenStepsInMeter, physicalDistanceBetweenLinesInMeter, picker["Steps"]["LargestGap"])

    print("LargestGap algoritmasına göre gitmesi gereken rota:")
    for i in range(len(picker["Steps"]["LargestGap"])):
        print("-> Adım " + str(i + 1) + " = ", picker["Steps"]["LargestGap"][i])

    print("")
    print("SShape algoritmasına göre gitmesi gereken yolun uzunluğu: " + str(sshapeDistance) + " metre.")
    print("MidPoint algoritmasına göre gitmesi gereken yolun uzunluğu: " + str(midpointDistance) + " metre.")
    print("LargestGap algoritmasına göre gitmesi gereken yolun uzunluğu: " + str(largestgapDistance) + " metre.")
    print("")
    print("# Sonuç")
    if sshapeDistance <= midpointDistance and sshapeDistance <= largestgapDistance:
        print(picker["Name"] + " personeline bulunan en kısa yol için SShape algoritması kullanılarak " + str(sshapeDistance) + " metre yol tercih edilmiştir.")
    elif midpointDistance <= sshapeDistance and midpointDistance <= largestgapDistance:
        print(picker["Name"] + " personeline bulunan en kısa yol için MidPoint algoritması kullanılarak " + str(midpointDistance) + " metre yol tercih edilmiştir.")
    else:
        print(picker["Name"] + " personeline bulunan en kısa yol için LargestGap algoritması kullanılarak " + str(largestgapDistance) + " metre yol tercih edilmiştir.")

    print("-----------------------------------------------------------------")

Gündüz 8 adet ürün toplayacaktır.

Ürünler:
-> Ürün 1 =  ProductNo: 1, Line: 1, BlockY: A, ChargeAddress: 5, BlockX: A, Category: B1,
-> Ürün 2 =  ProductNo: 2, Line: 1, BlockY: A, ChargeAddress: 11, BlockX: B, Category: B1,
-> Ürün 3 =  ProductNo: 3, Line: 1, BlockY: A, ChargeAddress: 26, BlockX: B, Category: B1,
-> Ürün 4 =  ProductNo: 6, Line: 2, BlockY: A, ChargeAddress: 24, BlockX: B, Category: B1,
-> Ürün 5 =  ProductNo: 9, Line: 3, BlockY: A, ChargeAddress: 19, BlockX: B, Category: B1,
-> Ürün 6 =  ProductNo: 19, Line: 4, BlockY: B, ChargeAddress: 36, BlockX: B, Category: GD1,
-> Ürün 7 =  ProductNo: 20, Line: 4, BlockY: B, ChargeAddress: 48, BlockX: B, Category: GD1,
-> Ürün 8 =  ProductNo: 21, Line: 4, BlockY: B, ChargeAddress: 54, BlockX: B, Category: GD1,

SShape algoritmasına göre gitmesi gereken rota:
-> Adım 1 =  MovL: 1, MovY: 16.0
-> Adım 2 =  MovL: 1, MovY: -16.0
-> Adım 3 =  MovL: 2, MovY: 16.0
-> Adım 4 =  MovL: 3, MovY: -6.0
-> Adım 5 =  MovL: 3, MovY: 6.0
-> Adım 6